In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from rtseg.utils.param_io import load_params
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
from rtseg.forkplot import compute_forkplot_stats
import matplotlib.patches as patches
from rtseg.dotdetect import compute_dot_coordinates
from rtseg.segmentation import LiveNet, get_live_model, live_segment
from rtseg.identify_channels import get_channel_locations
from scipy.signal import find_peaks
%matplotlib qt5

/home/pk/Documents/rtclient/.venv/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.2' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
params_path = Path('/home/pk/Documents/REALTIME/live_analysis_test/expt_params.yaml')
params = load_params(params_path, ref_type='expt')

In [4]:
data_dir = Path(params.Save.directory)

In [5]:
def get_dataponits(data_dir):
    all_images = []
    positions_dir = sorted(list(data_dir.glob('Pos*')), key=lambda x: int(x.name[3:]))
    for pos_dir in positions_dir:
        pos = int(pos_dir.name[3:])
        #print(pos)
        phase_img_filename = pos_dir / Path('phase/phase_0000.tiff')
        fluor_img_filename = pos_dir / Path('fluor/fluor_0000.tiff')
        datapoint = {}
        datapoint['phase'] = imread(phase_img_filename)
        datapoint['fluor'] = imread(fluor_img_filename)
        datapoint['position'] = pos
        datapoint['timepoint'] = 0
        all_images.append(datapoint)
    return all_images

In [6]:
all_data = get_dataponits(data_dir)
live_net = get_live_model(params)

In [12]:
params.BarcodeAndChannels.thresholds.conf=0.35

In [13]:
i = 0
datapoint = all_data[i]
seg_result = live_segment(datapoint, live_net, params)

In [14]:
seg_result.keys()

dict_keys(['phase', 'fluor', 'channel_mask', 'position', 'timepoint', 'barcode_locations', 'seg_mask', 'trap_locations_list', 'num_traps', 'error'])

In [16]:
for i in range(15, 30):
    datapoint = all_data[i]
    seg_result = live_segment(datapoint, live_net, params)
    fig, ax = plt.subplots(nrows=3, ncols=1)
    ax[0].imshow(seg_result['seg_mask'], cmap='viridis')
    ax[1].imshow(seg_result['phase'], cmap='gray')
    for row in seg_result['barcode_locations']:
        rect = patches.Rectangle((row[0], row[1]), row[2] - row[0], row[3] - row[1], linewidth=1, edgecolor='r', facecolor='none')
        ax[1].add_patch(rect)
    ax[2].imshow(seg_result['channel_mask'])
    plt.suptitle(f"Num traps: {seg_result['num_traps']}")
    print(i+1, '--->', len(seg_result['barcode_locations']))
    #image = seg_result['channel_mask']
    #hist = np.sum(image[400:800, :], axis = 0)
    #peaks, _ = find_peaks(hist, distance=65, prominence=200)
    #ax[3].plot(hist)
    #ax[3].plot(peaks, hist[peaks],'r*')
    plt.show()

16 ---> 3
17 ---> 3
18 ---> 3
19 ---> 3
20 ---> 3
21 ---> 3
22 ---> 3
23 ---> 4
24 ---> 3
25 ---> 3
26 ---> 3
27 ---> 3
28 ---> 3
29 ---> 3
30 ---> 2


In [13]:
image = seg_result['channel_mask']
hist = np.sum(image, axis = 0)
peaks, _ = find_peaks(hist, distance=65)
plt.figure()
plt.plot(hist)
plt.plot(peaks, hist[peaks],'r*')
plt.show()

In [17]:
a = np.array([1.0, 2.0, 1.0, 3.4, 2.0, 3.4, 1.0])
np.unique(a).tolist()

[1.0, 2.0, 3.4]

In [18]:
sorted(np.unique(a).tolist())

[1.0, 2.0, 3.4]